In [1]:
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'/Users/ryan/github/prosodic')
from llmdh import *
import prosodic
prosodic.USE_CACHE=False

In [2]:
df = pd.read_csv(PATH_KAGGLE).drop('Unnamed: 0',axis=1)
df = df.sample(frac=1)
df.iloc[0]

Author                                                      Charles Olson
Title                                            As the Dead Prey Upon Us
Poetry Foundation ID                                                54311
Content                 As the dead prey upon us,\nthey are the dead i...
Name: 838, dtype: object

In [3]:
@classmethod
def run_kaggle(self):
    with SqliteDict(PATH_OUTPUT,autocommit=True) as db, logmap('parsing') as lm, lm.verbosity(1):
        for i,row in lm(list(df.iterrows())):
            idx = row['Poetry Foundation ID']
            # if idx in db: continue
            poem_txt = row.Content
            # text = prosodic.Text(poem_txt, use_cache=False)
            db[idx] = {
                **dict(row),
                "num_stanzas": poem_txt.count('\n\n'),
                "num_lines": poem_txt.count('\n') - poem_txt.count('\n\n'),
                # "num_rhymes": len(text.get_rhyming_lines()),
                # "is_rhyming": text.is_rhyming,
            }

RhymeLLM.run_kaggle = run_kaggle
RhymeLLM.run_kaggle()

⎾ parsing @ 2024-03-15 13:49:32,505
￨ iterating: 100%|██████████| 15652/15652 [00:06<00:00, 2536.65it/s]
⎿ 6.5 seconds @ 2024-03-15 13:49:39,036


In [4]:
poem = prosodic.Text("""
Upon the stage of life, we play our parts,
As actors in a grand, unfolding tale,
Where love and fate entwine in countless arts,
And passions rise and wane, beyond the pale.

With quill in hand, we scribe our destiny,
In ink of tears and laughter's golden hue,
Each line a verse of sweet serenity,
Or tragic note that rends the heart in two.

Yet in this fleeting dance of joy and sorrow,
We find the beauty in the passing hour,
For every dawn brings forth a new tomorrow,
And every storm gives way to gentle shower.

So let us revel in this mortal fray,
And seize the moments ere they slip away.
""")

⎾ building text with 113 words @ 2024-03-15 14:22:38,951
￨ ⎾ tokenizing @ 2024-03-15 14:22:39,172
￨ ⎿ 0.1 seconds @ 2024-03-15 14:22:39,306
￨ ⎾ building stanzas @ 2024-03-15 14:22:39,306
￨ ￨ iterating stanzas: 100%|██████████| 4/4 [00:01<00:00,  2.58it/s]
￨ ⎿ 1.6 seconds @ 2024-03-15 14:22:40,867
⎿ 1.9 seconds @ 2024-03-15 14:22:40,867
⎾ saving object to RedisDict under key e85ae717 @ 2024-03-15 14:22:40,891
⎿ 0 seconds @ 2024-03-15 14:22:40,921


In [5]:
line=poem.lines[0]

In [6]:
parse=line.best_parse

⎾ parsing text Line(num=1, txt='Upon the stage of life, we play our parts,') @ 2024-03-15 14:23:54,596
￨ ⎾ saving object to RedisDict under key 6de42f95 @ 2024-03-15 14:23:54,810
￨ ⎿ 0 seconds @ 2024-03-15 14:23:54,837
⎿ 0.2 seconds @ 2024-03-15 14:23:54,838


In [28]:
def get_foot_type_results(txt):
    with logmap(announce=False) as lm, lm.verbosity(0):
        poem=prosodic.Text(txt)
        perc1=np.mean([
            int(line.best_parse.slots[3].meter_val=='s')
            for line in poem.lines
            if len(line.best_parse.slots)>=4
        ])
        perc2=np.mean([
            int(pos.meter_str=='ww')
            for line in poem.lines
            for pos in line.best_parse.positions
            if len(line.best_parse.slots)>=4
        ])
        return {
            'meter_perc_lines_fourthpos_s':perc1,
            'meter_mpos_ww':perc2    
        }


In [29]:
get_foot_type_results(
"""Upon the stage of life, we play our parts,
As actors in a grand, unfolding tale,
Where love and fate entwine in countless arts,
And passions rise and wane, beyond the pale.

With quill in hand, we scribe our destiny,
In ink of tears and laughter's golden hue,
Each line a verse of sweet serenity,
Or tragic note that rends the heart in two.

Yet in this fleeting dance of joy and sorrow,
We find the beauty in the passing hour,
For every dawn brings forth a new tomorrow,
And every storm gives way to gentle shower.

So let us revel in this mortal fray,
And seize the moments ere they slip away.
""")

{'meter_perc_lines_fourthpos_s': 1.0, 'meter_mpos_ww': 0.0}

In [11]:
slot=parse.slots[3]

In [12]:
slot.meter_val

's'